In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="{}".format(2) # gpu idx

In [2]:
import dlib
import cv2
import os
import re
import json
from pylab import *
from PIL import Image, ImageChops, ImageEnhance

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

/home/jiho/.venv/py369tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jiho/.venv/py369tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jiho/.venv/py369tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jiho/.venv/py369tf/lib/python3.6/site-packages/tensorflow/python/frame

In [4]:
#frame추출->png형식으로 저장(real, fake폴더에 각각 저장)
train_frame_folder = 'train_sample_videos'
with open(os.path.join(train_frame_folder, 'metadata.json'), 'r') as file:
    data = json.load(file)
list_of_train_data = [f for f in os.listdir(train_frame_folder) if f.endswith('.mp4')]
detector = dlib.get_frontal_face_detector()
for vid in list_of_train_data:
    count = 0
    cap = cv2.VideoCapture(os.path.join(train_frame_folder, vid))
    frameRate = cap.get(5)
    while cap.isOpened():
        frameId = cap.get(1)
        ret, frame = cap.read()
        if ret != True:
            break
        if frameId % ((int(frameRate)+1)*1) == 0:
            face_rects, scores, idx = detector.run(frame, 0)
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()
                crop_img = frame[y1:y2, x1:x2]
                if data[vid]['label'] == 'REAL':
                    cv2.imwrite('dataset/real/'+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))
                elif data[vid]['label'] == 'FAKE':
                    cv2.imwrite('dataset/fake/'+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))
                count+=1

"\ntrain_frame_folder = 'train_sample_videos'\nwith open(os.path.join(train_frame_folder, 'metadata.json'), 'r') as file:\n    data = json.load(file)\nlist_of_train_data = [f for f in os.listdir(train_frame_folder) if f.endswith('.mp4')]\ndetector = dlib.get_frontal_face_detector()\nfor vid in list_of_train_data:\n    count = 0\n    cap = cv2.VideoCapture(os.path.join(train_frame_folder, vid))\n    frameRate = cap.get(5)\n    while cap.isOpened():\n        frameId = cap.get(1)\n        ret, frame = cap.read()\n        if ret != True:\n            break\n        if frameId % ((int(frameRate)+1)*1) == 0:\n            face_rects, scores, idx = detector.run(frame, 0)\n            for i, d in enumerate(face_rects):\n                x1 = d.left()\n                y1 = d.top()\n                x2 = d.right()\n                y2 = d.bottom()\n                crop_img = frame[y1:y2, x1:x2]\n                if data[vid]['label'] == 'REAL':\n                    cv2.imwrite('dataset/real/'+vid.spl

In [5]:
input_shape = (128, 128, 3)
data_dir = 'dataset'

real_data = [f for f in os.listdir(data_dir+'/real') if f.endswith('.png')]
fake_data = [f for f in os.listdir(data_dir+'/fake') if f.endswith('.png')]

X = []
Y = []

for img in real_data:
    X.append(img_to_array(load_img(data_dir+'/real/'+img)).flatten() / 255.0)
    Y.append(1)
for img in fake_data:
    X.append(img_to_array(load_img(data_dir+'/fake/'+img)).flatten() / 255.0)
    Y.append(0)

Y_val_org = Y

#Normalization
X = np.array(X)
Y = to_categorical(Y, 2)

#Reshape
X = X.reshape(-1, 128, 128, 3)

#Train-Test split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)

In [6]:
from tensorflow.keras.applications import InceptionResNetV2
#from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

resNet_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
resNet_model.trainable = True
model = Sequential()
model.add(resNet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(units=2, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
optimizer=optimizers.Adam(lr=1e-5, epsilon=None)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 2, 2, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 3074      
Total params: 54,339,810
Trainable params: 54,279,266
Non-trainable params: 60,544
_________________________________________________________________


In [8]:
import keras
callbacks = [
    keras.callbacks.ModelCheckpoint(
        '/home/jiho/work/repos/deepfake/model/weights.{epoch:02d}-{val_loss:.2f}.hdf5', 
        verbose=1),
    keras.callbacks.TensorBoard(log_dir='/home/jiho/work/repos/deepfake')#그래프로 보기위해 기록 저장
]

Using TensorFlow backend.


In [9]:
#Currently not used
EPOCHS = 20
BATCH_SIZE = 100
history = model.fit(X_train, Y_train, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_data = (X_val, Y_val), callbacks=callbacks)

Train on 2996 samples, validate on 749 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
2900/2996 [============================>.] - ETA: 1s - loss: 0.5468 - acc: 0.7817
Epoch 00001: saving model to /home/jiho/work/repos/deepfake/model/weights.01-0.61.hdf5
2996/2996 [==============================] - 45s 15ms/sample - loss: 0.5444 - acc: 0.7830 - val_loss: 0.6052 - val_acc: 0.7664
Epoch 2/20
2900/2996 [============================>.] - ETA: 0s - loss: 0.4173 - acc: 0.8417
Epoch 00002: saving model to /home/jiho/work/repos/deepfake/model/weights.02-0.55.hdf5
2996/2996 [==============================] - 12s 4ms/sample - loss: 0.4163 - acc: 0.8435 - val_loss: 0.5456 - val_acc: 0.7850
Epoch 3/20
2900/2996 [============================>.] - ETA: 0s - loss: 0.3353 - acc: 0.8817
Epoch 00003: saving model to /home/jiho/work/repos/deepfake/model/weights.03-0.51.hdf5
2996/2996 [==============================] - 12s 4ms/sample - loss: 0.3334 - acc: 0.8842 - val_loss: 0.5091 - va